## Create parameters file

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List

from hscpy import mitchell, realisation, parameters
from futils import parse_version

NCELLS = 100_000
REPLICATES = 500
PATH2HSC = Path("~").expanduser() / "hsc"
PATH2DATA = Path("~").expanduser() / "hsc-draft/data"

In [ ]:
%%bash -s "$PATH2HSC" --out version
$1/target/release/hsc  --version

In [ ]:
VERSION = parse_version(version)
PATH2SAVE = Path(f"./{VERSION}")

print("Running hsc with version:", VERSION)

In [ ]:
donors = mitchell.donors()
donors

In [ ]:
def create_parameters(
    donors: pd.DataFrame,
    names: List[str],
    eta: float,
    sigma: float,
    mu: int,
    mu_exp: int,
    tau: float,
    exp_phase: bool = True,
    dir2save: Path = PATH2DATA / VERSION,
) -> realisation.SimulationCMD:
    return realisation.SimulationCMD(
        cells=NCELLS,
        samples=donors[donors.name.isin(names)].cells.tolist(),
        eta=eta,
        sigma=sigma,
        mu=mu,
        mu_exp=mu_exp,
        tau=tau,
        ages=donors[donors.name.isin(names)].age.tolist(),
        name=name,
        seed=None,
        exp_phase=exp_phase,
        tau_exp=parameters.tau_exp(NCELLS),
        dir2save=dir2save.absolute(),
    )

In [ ]:
# params are from ABC
parameters_ = dict()

name = "CB001"
eta, sigma, mu, tau = 0.03, 0.01, 1, 1
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

# assume same params for both 0 age donors
name = "CB002"
eta, sigma, mu, tau = 0.03, 0.01, 1, 1
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

name = "KX001"
eta, sigma, mu, tau = 0.03, 0.01, 1, 1.8
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

name = "KX002"
eta, sigma, mu, tau = 0.02, 0.01, 1, 2.6
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

name = "SX001"
eta, sigma, mu, tau = 0.03, 0.01, 1, 2.6
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

name = "AX001"
eta, sigma, mu, tau = 0.02, 0.01, 1, 3
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

name = "KX008"
eta, sigma, mu, tau = 0.06, 0.04, 16, 5
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

name = "KX004"
eta, sigma, mu, tau = 0.12, 0.04, 16, 1.4
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

name = "KX003"
eta, sigma, mu, tau = 0.08, 0.04, 23, 3
parameters_[name] = create_parameters(
    donors,
    [name],
    eta,
    sigma,
    mu,
    mu,
    tau,
    dir2save=(PATH2DATA / Path(f"simulations/{VERSION}")),
)

In [ ]:
def write_parameters_to_file(
    parameters: List[realisation.SimulationCMD], path2file: Path, mode="w"
):
    with open(path2file, mode) as f:
        for params in parameters:
            for _ in range(REPLICATES):
                f.write(params.parameters() + "\n")
    assert path2file.is_file()

In [ ]:
write_parameters_to_file(list(parameters_.values()), Path("parameters.txt"))

In [ ]:
%%bash
head parameters.txt

In [ ]:
%%bash
tail parameters.txt

In [ ]:
%%bash
wc -l parameters.txt

In [ ]:
%%bash --out runs
wc -l parameters.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert (
    REPLICATES * len(parameters_) == runs
), f"{REPLICATES * len(parameters_)} vs {runs}"

In [ ]:
REPLICATES * len(parameters_)

## The parameters for dynamics

In [ ]:
names = donors.name.unique()
path2dir = PATH2DATA / f"dynamics/{VERSION}"

parameters_varying_eta = list()
etas, sigma, mu, tau = [0.05, 0.08, 0.15], 0.01, 1, 1
for eta in etas:
    parameters_varying_eta.append(
        create_parameters(
            donors,
            names,
            eta,
            sigma,
            mu,
            mu,
            tau,
            exp_phase=True,
            dir2save=path2dir / "etas",
        )
    )

parameters_varying_mu = list()
eta, sigma, mus, tau = 0.08, 0.01, [0.5, 3, 10], 1
for mu in mus:
    parameters_varying_mu.append(
        create_parameters(
            donors,
            names,
            eta,
            sigma,
            mu,
            mu,
            tau,
            exp_phase=True,
            dir2save=path2dir / "mus",
        )
    )


parameters_varying_tau = list()
eta, sigma, mu, taus = 0.08, 0.01, 1, [0.3, 1, 5]
for tau in taus:
    parameters_varying_tau.append(
        create_parameters(
            donors,
            names,
            eta,
            sigma,
            mu,
            mu,
            tau,
            exp_phase=True,
            dir2save=path2dir / "taus",
        )
    )

In [ ]:
for file, params in zip(
    [
        "parameters_varying_eta.txt",
        "parameters_varying_mu.txt",
        "parameters_varying_tau.txt",
    ],
    [parameters_varying_eta, parameters_varying_mu, parameters_varying_tau],
):
    print(file)
    write_parameters_to_file(params, Path(file), mode="w")

In [ ]:
%%bash
head parameters_varying_mu.txt

In [ ]:
%%bash
tail parameters_varying_mu.txt

In [ ]:
%%bash
tail parameters_varying_eta.txt

In [ ]:
%%bash --out runs
wc -l parameters_varying_eta.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert (
    REPLICATES * len(parameters_varying_eta) == runs
), f"{REPLICATES * len(parameters_varying_eta)} vs {runs}"

REPLICATES * len(parameters_varying_eta)

## The parameters for sde

In [ ]:
path2dir = PATH2DATA / f"sde/{VERSION}"
donors = pd.DataFrame(
    [
        (n, a, c)
        for n, a, c in zip(
            [f"t{i}" for i in range(0, 21)], range(0, 105, 5), [NCELLS] * 100
        )
    ],
    columns=["name", "age", "cells"],
)
names = donors.name.squeeze()
donors

In [ ]:
parameters_varying_eta1 = list()
etas, sigma, mu, tau = [0.05, 0.2], 0.025, 4, 1
for eta in etas:
    parameters_varying_eta1.append(
        create_parameters(
            donors,
            names,
            eta,
            sigma,
            mu,
            0,
            tau,
            exp_phase=False,
            dir2save=path2dir / "etas_mu4",
        )
    )

parameters_varying_eta2 = list()
etas, sigma, mu, tau = [0.05, 0.2], 0.025, 15, 1
for eta in etas:
    parameters_varying_eta2.append(
        create_parameters(
            donors,
            names,
            eta,
            sigma,
            mu,
            0,
            tau,
            exp_phase=False,
            dir2save=path2dir / "etas_mu15",
        )
    )

In [ ]:
for file, params in zip(
    [
        "parameters_sde_varying_eta1.txt",
        "parameters_sde_varying_eta2.txt",
    ],
    [parameters_varying_eta1, parameters_varying_eta2],
):
    print(file)
    write_parameters_to_file(params, Path(file), mode="w")

In [ ]:
%%bash --out runs
wc -l parameters_sde_varying_eta1.txt

In [ ]:
runs = int(runs.split(" ")[0])
assert (
    REPLICATES * len(parameters_varying_eta1) == runs
), f"{REPLICATES * len(parameters_varying_eta1)} vs {runs}"

REPLICATES * len(parameters_varying_eta1)

In [ ]:
%%bash
tail parameters_sde_varying_eta1.txt

In [ ]:
%%bash
tail parameters_sde_varying_eta2.txt